Nejprve jsem vyčistila soubor 02_gls_raw_all.json, který jsem uložila do  
pomocného csv, kam jsem vložila RUIAN kód obce. V druhém běhu čištění dat  
tedy otevrírám tento pomocný soubor, ve kterém jsou přidány sloupce z databáze  
adres.

In [ ]:
import pandas as pd
data = pd.read_csv('gls_pomocny_dataset_kod_obce.csv', dtype={'city_code': str, 'zipcode': str})

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4407 entries, 0 to 4406
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   address           4407 non-null   object 
 1   pclshopid         4407 non-null   object 
 2   _leaflet_id       4407 non-null   object 
 3   ctrcode           4407 non-null   object 
 4   city              4407 non-null   object 
 5   geolat            4407 non-null   float64
 6   geolng            4407 non-null   float64
 7   iscodhandler      4407 non-null   int64  
 8   isparcellocker    4407 non-null   int64  
 9   contact           0 non-null      float64
 10  phone             0 non-null      float64
 11  name              4407 non-null   object 
 12  guid              4407 non-null   object 
 13  visible           4407 non-null   int64  
 14  utilization       4407 non-null   int64  
 15  capacitystate     4407 non-null   int64  
 16  parcelBox         4407 non-null   int64  


Obecné poznámky k datasetu:  
Parcel boxy: 2720 míst  
Parcel shopy: 1687 míst  
Sloupce s 1 hodnotou: 'ctrcode', 'iscodhandler', 'contact', 'phone', 'visible'  
Sloupce: parcelBox a isparcellocker	jsou totožné, Parcelbox/ParcelShop  

Dotazy, připomínky:  
1. 
2. 

1. EXPLORACE DATASETU v DF ale i v pomocném .CSV souboru (pro lepší přehled).
   Pomocný soubor v procesu průběžně přepisuji.

In [114]:
df = pd.DataFrame(data)

df.head(2)

,address,pclshopid,_leaflet_id,ctrcode,city,geolat,geolng,iscodhandler,isparcellocker,contact,...,wheelchairAccess,zipcode,openSaturday,openSunday,imageUrl,holiday,city_code,city_name,search_term,search_type
0,S. K. Neumanna 590/22,CZ58601-PARCELLOCK01,CZ58601-PARCELLOCK01,CZ,Jihlava,49.4048,15.557,1,1,NaN,...,NaN,58601,1,1,https://prodpdmpublicstore.blob.core.windows.n...,NaN,586846.0,Jihlava,S. K. Neumanna 590/22 Jihlava 58601,full
1,Myslotínská 2449,39301-ELPESRO,39301-ELPESRO,CZ,Pelhřimov,49.4139,15.220,1,0,NaN,...,NaN,39301,1,0,https://prodpdmpublicstore.blob.core.windows.n...,NaN,547492.0,Pelhřimov,Myslotínská 2449 Pelhřimov 39301,full


In [115]:
df.shape

(4407, 30)

Výskyt a počet Nan - pouze COL 'holiday'

In [116]:
df.isnull().sum()

address                0
pclshopid              0
_leaflet_id            0
ctrcode                0
city                   0
geolat                 0
geolng                 0
iscodhandler           0
isparcellocker         0
contact             4407
phone               4407
name                   0
guid                   0
visible                0
utilization            0
capacitystate          0
parcelBox              0
parcelShop             0
payCard                0
paybybankcard          0
wheelchairAccess    3524
zipcode                0
openSaturday           0
openSunday             0
imageUrl             772
holiday             4161
city_code             40
city_name             40
search_term           40
search_type           40
dtype: int64

přehled kategorií/textových sloupců:

In [117]:
df.select_dtypes(include='object').nunique()

address        4363
pclshopid      4407
_leaflet_id    4407
ctrcode           1
city           1367
name           3310
guid           4389
zipcode        1138
imageUrl       3627
holiday          51
city_code      1033
city_name      1016
search_term    4242
search_type       4
dtype: int64

sloupce, seřazené dle hodnot

In [118]:
df.nunique().sort_values()

phone                  0
contact                0
ctrcode                1
iscodhandler           1
visible                1
wheelchairAccess       1
parcelBox              2
isparcellocker         2
parcelShop             2
openSunday             2
openSaturday           2
paybybankcard          2
payCard                2
capacitystate          3
search_type            4
utilization           36
holiday               51
city_name           1016
city_code           1033
zipcode             1138
city                1367
name                3310
imageUrl            3627
search_term         4242
geolat              4263
geolng              4319
address             4363
guid                4389
_leaflet_id         4407
pclshopid           4407
dtype: int64

Všechny sloupce, které obsahují jen 1 unikátní hodnotou, odstranit:

In [119]:
df.columns[df.nunique() == 1]

Index(['ctrcode', 'iscodhandler', 'visible', 'wheelchairAccess'], dtype='object')

In [120]:
df = df.drop(columns = df.columns[df.nunique()==1])

počty výskytů konkrétních hodnot v jednom sloupci:

In [121]:
df['city'].value_counts()

city
Praha               167
Brno                111
Ostrava              82
Plzeň                82
České Budějovice     58
                   ... 
Nesvačilka            1
Bedihošť              1
Rudíkov               1
Hošťálkovy            1
Nepřevázka            1
Name: count, Length: 1367, dtype: int64

In [122]:
df['isparcellocker'].value_counts()


isparcellocker
1    2720
0    1687
Name: count, dtype: int64

In [123]:
df['parcelBox'].value_counts()

parcelBox
1    2720
0    1687
Name: count, dtype: int64

In [124]:
df.columns[df.nunique() != 1]

Index(['address', 'pclshopid', '_leaflet_id', 'city', 'geolat', 'geolng',
       'isparcellocker', 'contact', 'phone', 'name', 'guid', 'utilization',
       'capacitystate', 'parcelBox', 'parcelShop', 'payCard', 'paybybankcard',
       'zipcode', 'openSaturday', 'openSunday', 'imageUrl', 'holiday',
       'city_code', 'city_name', 'search_term', 'search_type'],
      dtype='object')

2. ROWS - ve sloupci ISPARCELLOCKER  
VYBRAT DO DF pouze boxy  
Sloupec  ISPARCELLOCKER 1 = BOX,  0 = výdejní místo nebo provozovna, depo  

In [125]:
boxy = df[df['isparcellocker'] == 1]

In [126]:
df = boxy
df.shape

(2720, 26)

3. COLUMNS - Vybrat jen nezbytné sloupce, ostatní zahodit pro lepší přehled dat.
   Seřadit sloupce

In [127]:
df.columns

Index(['address', 'pclshopid', '_leaflet_id', 'city', 'geolat', 'geolng',
       'isparcellocker', 'contact', 'phone', 'name', 'guid', 'utilization',
       'capacitystate', 'parcelBox', 'parcelShop', 'payCard', 'paybybankcard',
       'zipcode', 'openSaturday', 'openSunday', 'imageUrl', 'holiday',
       'city_code', 'city_name', 'search_term', 'search_type'],
      dtype='object')

In [128]:
df = df[['name', 'city_code', 'city_name', 'city', 'zipcode', 'address', 'geolat', 'geolng']]
df.head(3)

,name,city_code,city_name,city,zipcode,address,geolat,geolng
0,GLS Parcel Box,586846.0,Jihlava,Jihlava,58601,S. K. Neumanna 590/22,49.4048,15.5570
743,GLS Parcel Box,597007.0,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská 147/25,49.3517,16.0196
747,GLS Parcel Box,545058.0,Valašské Meziříčí,Valašské Meziříčí,75701,Králova 1256,49.4693,17.9711


In [129]:
df.columns

Index(['name', 'city_code', 'city_name', 'city', 'zipcode', 'address',
       'geolat', 'geolng'],
      dtype='object')

4. ROWS in NAME  
   DF obsahuje boxy od 3. stran AlzaBox a AlzaWall (to je v Alzaprodejně)  
   po odstranění zbyde 973 záznamů  
   | - regex nebo

In [130]:
alza_boxwall = df[df['name'].str.contains(r'AlzaBox|AlzaWall', na=False)]
alza_boxwall.shape

(1747, 8)

In [131]:
df = df[~df['name'].str.contains(r'AlzaBox|AlzaWall', na=False)]
df.shape

(973, 8)

5. COL - NAME  
   odstranit (*)  
např. GLS Parcel Box (z ulice Matějkova)

In [132]:
name_se_zavorkou= df[df['name'].str.contains(r'\(.*\)$', na=False)]
name_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng
752,GLS Parcel Box (NC Haná),500496.0,Olomouc,Olomouc,77900,Kafkova 1223/8 (OC Haná),49.575400,17.228200
763,GLS Parcel Box (Lékárna u sv. Anny),582786.0,Brno,Brno,60200,Mendlovo náměstí,49.190446,16.596150
782,GLS Parcel Box (NC Fénix),554782.0,Praha,Praha,19000,Freyova 945,50.109900,14.501600
800,GLS Parcel Box (Sportovní hala Lužiny),554782.0,Praha,Praha,15500,Bellušova 1877/68,50.041100,14.325200
822,GLS Parcel Box (OC Atrium),569810.0,Hradec Králové,Hradec Králové,50002,Dukelská třída 1713/7,50.213200,15.818800
837,GLS Parcel Box (Homepark Zličín),554782.0,Praha,Praha,15500,Skandinávská 144/25,50.050118,14.294846
841,GLS Parcel Box (NORMA),539333.0,Jílové u Prahy,Jílové u Prahy,25401,Rudných dolů 502,49.896700,14.492400
843,GLS Parcel Box (Menza ČVUT),554782.0,Praha,Praha,16000,Kolejní 2571,50.105594,14.389182
847,GLS Parcel Box (OMV),554821.0,Ostrava,Ostrava,70200,Mariánskohorská 3121/55,49.840600,18.260400
872,GLS Parcel Box (OC Futurum),582786.0,Brno,Brno,61900,Vídeňská 132/100,49.155800,16.597600


In [133]:
df.loc[:, 'name'] = df['name'].str.replace(r'\s*\(.*\)$', '', regex=True)
name_se_zavorkou= df[df['name'].str.contains(r'\(.*\)$', na=False)]
name_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng


6. COL - city_code  
    odstranit .0 za číslem

In [134]:
df.loc[:, 'city_code'] = df['city_code'].str.replace(r'\.0$', '', regex=True)

In [135]:
df.head(3)

,name,city_code,city_name,city,zipcode,address,geolat,geolng
0,GLS Parcel Box,586846,Jihlava,Jihlava,58601,S. K. Neumanna 590/22,49.4048,15.5570
743,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská 147/25,49.3517,16.0196
747,GLS Parcel Box,545058,Valašské Meziříčí,Valašské Meziříčí,75701,Králova 1256,49.4693,17.9711


In [136]:
df.to_csv('gls_pomocny_dataset.csv', index=False, encoding='utf-8')

7. COLUMNS - city  
   př. chyb:  
Ceske Budejovice 2 (Ceske  
Breclav (Breclav)  
Brno-Bohunice X) (Brno-Me  
Brno-Cernovice X) (Brno-M  
Brno-Kohoutovice X) (Brno  
Brno-Lisen X)  
Blansko (Blansko)  
Hodonin (Hodonin)  
Chomutov (Chomutov)  
Kladno (Kladn)  
Kladno (Kladn)  
Kladno (Kladn)  
Kunovice (Uherske Hradist  
Moravske Budejovice (Treb  
Nelze paušálně odstranit "vše v závorce", ne všechny závorky jsou špatně

In [137]:
df['city'] = df['city'].str.strip().str.title()

In [138]:
city_Xrozbite= df[df['city'].str.contains(r'\s*X\).*', na=False)]

city_Xrozbite

,name,city_code,city_name,city,zipcode,address,geolat,geolng
2058,GLS Parcel Box,582786,Brno,Brno-Slatina X),62700,Tuřanka 1260/1,49.176424,16.688113
2265,GLS Parcel Box,554821,Ostrava,Ostrava-Zabreh X),70030,Výškovická 2636/116,49.790598,18.227767
2356,GLS Parcel Box,582786,Brno,Brno-Bohunice X) (Brno-Me,63900,Vídeňská 815/89a,49.175910,16.597449
2357,GLS Parcel Box,582786,Brno,Brno-Kohoutovice X) (Brno,62300,Libušina tř. 580/4,49.197138,16.533717
2415,GLS Parcel Box,582786,Brno,Brno-Lisen X),62800,Sedláčkova 2557/4,49.202316,16.668318
2620,GLS Parcel Box,554821,Ostrava,Ostrava-Poruba X),70800,Pavlouskova 4457/24,49.838772,18.163377
2621,GLS Parcel Box,554821,Ostrava,Ostrava-Poruba X),70800,Oty Synka 1876/8,49.845491,18.160582
3913,GLS Parcel Box,NaN,NaN,Brno-Cernovice X) (Brno-M,61800,Blatouchová 1074/2b,49.185489,16.632651


In [139]:
df['city'] = df['city'].str.replace(r'\s*X\).*', '', regex=True)

In [140]:
city_Xrozbite= df[df['city'].str.contains(r'\s*X\).*', na=False)]
city_Xrozbite

,name,city_code,city_name,city,zipcode,address,geolat,geolng


In [141]:
df['shoda_city'] = df['city'] == df['city_name']

# Výběr jen těch řádků, kde se město neshoduje
neshody = df[df['shoda_city'] == False]

# Výpis prvních několika neshod (volitelně)
neshody[['city', 'city_name', 'shoda_city']].head()

,city,city_name,shoda_city
841,Jílové U Prahy,Jílové u Prahy,False
1516,Frýdek Místek,Frýdek-Místek,False
1519,Frýdek Místek,Frýdek-Místek,False
1548,Ústí Nad Labem,Ústí nad Labem,False
1855,Chomutov,Otvice,False


In [142]:
city_se_zavorkou = df[df['city'].str.contains(r'\(.*\)$', na=False)]

city_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city
1985,GLS Parcel Box,552046,Tábor,Tabor (Tabor),39005,Helsinská 2760/18,49.385556,14.688058,False
2070,GLS Parcel Box,562971,Chomutov,Chomutov (Chomutov),43004,Pod Břízami 5251,50.491305,13.438580,False
2118,GLS Parcel Box,567027,Most,Most (Most),43401,Rudolická 1706/4,50.510646,13.651240,False
2167,GLS Parcel Box,532053,Kladno,Kladno (Kladn),27201,Americká 3321,50.128726,14.116330,False
2169,GLS Parcel Box,532053,Kladno,Kladno (Kladn),27201,Náměstí Sítná 3127,50.135393,14.102904,False
2244,GLS Parcel Box,579025,Trutnov,Vnitrni Mesto (Trutnov),54101,Na Struze 30,50.562256,15.918045,False
2246,GLS Parcel Box,581283,Blansko,Blansko (Blansko),67801,Poříčí 2610,49.368950,16.640350,False
2249,GLS Parcel Box,534676,Mělník,Melnik (Melnik),27601,Vodárenská 219,50.352256,14.480146,False
2250,GLS Parcel Box,532053,Kladno,Kladno (Kladn),27201,Hřebečská 2732,50.132740,14.137471,False
2306,GLS Parcel Box,585599,Otrokovice,Otrokovice (Zlin),76502,Příčná 1580,49.203706,17.538262,False


In [154]:
mask = df['city'].str.contains(r'\(.*\)$', na=False)
df.loc[mask, 'city'] = df.loc[mask, 'city_name']

In [155]:
df[df['city'].str.contains(r'\(.*\)$', na=False)]

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id


In [156]:
df.to_csv('gls_pomocny_dataset.csv', index=False, encoding='utf-8')

!!!! Sem se vrátím a dočistím později. Potřebuji promyslet. Není správné 
odstranit všechny závorky paušálně. V datasetu vidím, že může být adresa zapsána
různě. Je třeba se poradit.

8. Address

In [146]:
adresy_se_zavorkou = df[df['address'].str.contains(r'\(.*\)$', na=False)]

adresy_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city
752,GLS Parcel Box,500496,Olomouc,Olomouc,77900,Kafkova 1223/8 (OC Haná),49.575400,17.228200,True
761,GLS Parcel Box,555088,Havířov,Havířov,73601,Dělnická (OC Elan),49.780800,18.431800,True
766,GLS Parcel Box,554821,Ostrava,Ostrava,72527,Spojovací 396 (NC Géčko),49.849000,18.145800,True
776,GLS Parcel Box,544256,České Budějovice,České Budějovice,37001,Nádražní 1759 (OC Mercury),48.972900,14.487400,True
777,GLS Parcel Box,582786,Brno,Brno,62500,Netroufalky 770/14-16 (OC Campus Square),49.177700,16.566800,True
838,GLS Parcel Box,554782,Praha,Praha,16900,Chaloupeckého 1914/9 (hřiště proti bloku 4),50.080300,14.392800,True
2358,GLS Parcel Box,554782,Praha,Praha,17000,Veletržní 1623/24 (-1P Garáže),50.101736,14.430196,True
2912,GLS Parcel Box,582786,Brno,Brno Židenice,61500,Bubeníčkova (stará osada),49.201776,16.641489,False
3069,GLS Parcel Box,554782,Praha,Praha,14000,Duhová 1531/3 (zadní strana parkoviště),50.049215,14.450828,True
3128,GLS Parcel Box,572926,Hořice,Hořice,50801,Škroupova (vedle jednoty),50.367666,15.635367,True


In [147]:
df.loc[:, 'address'] = df['address'].str.replace(r'\s*\(.*\)$', '', regex=True)
adresy_se_zavorkou = df[df['address'].str.contains(r'\(.*\)$', na=False)]

adresy_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city


In [159]:
adresy_s_cislem = df[df['address'].str.contains(r'\s\d[\dA-Za-z/,\- ]*(?:/[IVXLCDM]+)?$', na=False)]

adresy_s_cislem

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id
0,GLS Parcel Box,586846,Jihlava,Jihlava,58601,S. K. Neumanna 590/22,49.404800,15.557000,True,1
743,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská 147/25,49.351700,16.019600,True,2
747,GLS Parcel Box,545058,Valašské Meziříčí,Valašské Meziříčí,75701,Králova 1256,49.469300,17.971100,True,3
748,GLS Parcel Box,554782,Praha,Praha,15600,Pod Třešňovkou 234,49.970800,14.383200,True,4
749,GLS Parcel Box,511382,Přerov,Přerov,75002,Svisle 1100/9,49.450300,17.459400,True,5
...,...,...,...,...,...,...,...,...,...,...
4271,GLS Parcel Box,585068,Zlín,Zlín,76001,Ladislava Váchy 517,49.217261,17.633535,True,969
4272,GLS Parcel Box,582786,Brno,Brno,62700,Langrova 1227/1d,49.179369,16.679533,True,970
4301,GLS Parcel Box,596230,Nové Město na Moravě,Nové Město Na Moravě,59231,Komenského náměstí 999,49.563083,16.075644,False,971
4309,GLS Parcel Box,583782,Rosice,Rosice U Brna,66501,Zastávecká 1267,49.184816,16.368091,False,972


In [160]:
adresy_bez_cisla = df[~df['address'].str.contains(r'\s\d[\dA-Za-z/,\- ]*(?:/[IVXLCDM]+)?$', na=False)]

adresy_bez_cisla

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id
761,GLS Parcel Box,555088,Havířov,Havířov,73601,Dělnická,49.780800,18.431800,True,9
763,GLS Parcel Box,582786,Brno,Brno,60200,Mendlovo náměstí,49.190446,16.596150,True,10
796,GLS Parcel Box,554782,Praha,Praha,14000,Na Hřebenech II,50.053585,14.430863,True,27
1531,GLS Parcel Box,554821,Ostrava,Ostrava,70030,Dr. Lukášové křižovatka Holasova,49.787700,18.256500,True,78
1909,GLS Parcel Box,536385,Jeseník,Jeseník,79001,Autobusové n.-Smetanova ul.,50.227500,17.208300,True,118
2375,GLS Parcel Box,592731,Uherský Brod,Uherský Brod,68801,Močidla,49.025775,17.660271,True,284
2649,GLS Parcel Box,587478,Luka nad Jihlavou,Luka Nad Jihlavou,58822,Dělnická,49.377473,15.698793,False,331
2650,GLS Parcel Box,546127,Dačice,Dačice,38001,Pantočkova,49.080330,15.430082,True,332
2659,GLS Parcel Box,534951,Kralupy nad Vltavou,Kralupy Nad Vltavou,27801,Libušina ulice,50.240607,14.317116,False,341
2864,GLS Parcel Box,553271,Veselí nad Lužnicí,Veselí Nad Lužnicí,39181,Tř. Československé Armády 609 - u myček,49.197283,14.716687,False,388


In [162]:
df.loc[:,'house_no'] = df['address'].str.extract(r'(\d[\dA-Za-z/,\- ]*(/[IVXLCDM]+)?)$')[0]

df.head(2)

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id,house_no
0,GLS Parcel Box,586846,Jihlava,Jihlava,58601,S. K. Neumanna 590/22,49.4048,15.5570,True,1,590/22
743,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská 147/25,49.3517,16.0196,True,2,147/25


In [163]:
df.loc[:,'address'] = df['address'].str.replace(r'\s\d[\dA-Za-z/,\- ]*(/[IVXLCDM]+)?$', '', regex = True)

df.head(2)

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id,house_no
0,GLS Parcel Box,586846,Jihlava,Jihlava,58601,S. K. Neumanna,49.4048,15.5570,True,1,590/22
743,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská,49.3517,16.0196,True,2,147/25


In [164]:
adresy_s_pomlckou = df[df['address'].str.contains(r'-', na=False)]

df.columns
adresy_s_pomlckou

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id,house_no
1909,GLS Parcel Box,536385,Jeseník,Jeseník,79001,Autobusové n.-Smetanova ul.,50.227500,17.208300,True,118,NaN
2864,GLS Parcel Box,553271,Veselí nad Lužnicí,Veselí Nad Lužnicí,39181,Tř. Československé Armády 609 - u myček,49.197283,14.716687,False,388,NaN
2870,GLS Parcel Box,588458,Holešov,Holešov,76901,Obchodní galerie - ulice Tovární,49.327873,17.573918,True,393,511
2960,GLS Parcel Box,578347,Litomyšl,Litomyšl,57001,Sokolovská 113 - za automyčkou,49.880838,16.292915,True,420,NaN
3096,GLS Parcel Box,561380,Česká Lípa,Česká Lípa,47001,Na Blatech - automyčka DIBO,50.687907,14.519294,True,495,NaN
3106,GLS Parcel Box,573868,Náchod,Náchod,54701,Mlýnská - parkoviště,50.414193,16.162277,True,505,NaN
3133,GLS Parcel Box,576069,Rychnov nad Kněžnou,Rychnov Nad Kněžnou,51601,Zborovská 17 - vedle katastrálního úřadu,50.163417,16.278833,False,519,NaN
3207,GLS Parcel Box,554782,Praha,Praha,14700,Ke Krči - zastávka autobusu Branické náměstí,50.034667,14.414569,True,549,NaN
3466,GLS Parcel Box,573248,Nová Paka,Nová Paka,50901,Na Tržišti 23 - Parkoviště,50.494030,15.515213,True,645,NaN
3532,GLS Parcel Box,554782,Praha,Praha 20 Horní Počernice,19300,Pavlišovská 2292/14 - vjezd z ulice Jizbická,50.110422,14.621840,False,676,NaN


In [167]:
specificke_adresy = df[df['address'].str.contains(' - parkoviště| - u myček| - Parkoviště| - za automyčkou| - vedle katastrálního úřadu| - vjezd z ulice Jizbická', na=False)]

df.loc[:,'address'] = df['address'].str.replace(' - parkoviště| - u myček| - Parkoviště| - za automyčkou| - vedle katastrálního úřadu| - vjezd z ulice Jizbická', '', regex = True)


In [168]:
specificke_adresy

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id,house_no


In [169]:
df.head(3)

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,id,house_no
0,GLS Parcel Box,586846,Jihlava,Jihlava,58601,S. K. Neumanna,49.4048,15.5570,True,1,590/22
743,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská,49.3517,16.0196,True,2,147/25
747,GLS Parcel Box,545058,Valašské Meziříčí,Valašské Meziříčí,75701,Králova,49.4693,17.9711,True,3,1256


Vytvořit sloupec ID  
Založit unikátní čísla, přepíšou se pro celý dataset boxů v SQL

In [148]:
df['id'] = range(1, len(df) + 1)

In [175]:
df.loc[:,'house_no'] = df['house_no'].astype(str)

Finální přejmenování a seřazení sloupců
ID 	name	city_code	city_code_name	city	zip_code	street	house_NO	latitude 	longtitude

In [ ]:
df = df.rename(columns={
    'address': 'street'
  })

df = df[['id', 'name', 'city_code', 'city_name', 'city', 'shoda_city', 'zipcode', 'street', 'house_no',  'geolat', 'geolng']]
df.head(3)

In [173]:
df = df[['id', 'name', 'city_code', 'city_name', 'city', 'shoda_city', 'zipcode', 'street', 'house_no',  'geolat', 'geolng']]
df.head(3)

,id,name,city_code,city_name,city,shoda_city,zipcode,street,house_no,geolat,geolng
0,1,GLS Parcel Box,586846,Jihlava,Jihlava,True,58601,S. K. Neumanna,590/22,49.4048,15.5570
743,2,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,True,59401,Sokolovská,147/25,49.3517,16.0196
747,3,GLS Parcel Box,545058,Valašské Meziříčí,Valašské Meziříčí,True,75701,Králova,1256,49.4693,17.9711


In [176]:
df['house_no'] = df['house_no'].astype(str)
df['geolat'] = df['geolat'].astype(float)
df['geolng'] = df['geolng'].astype(float)

In [178]:
df = df.drop(columns=['shoda_city'])

In [179]:
df.to_csv('gls_pomocny_dataset.csv', index=False, encoding='utf-8')

In [151]:
# df['city'] = df['city'].str.replace(r'\s*\((\s*\1)?\)', '', regex=True).str.strip()

Finální dataset

In [152]:
# df.to_csv('ss_clean_data_gls.json', index=False, encoding='utf-8')

In [153]:
# import json

# df.to_json('ss_clean_data_gls.json', orient='records', force_ascii=False, indent=4)